In [0]:
# Installing all the libraries
!pip install pandas
!pip install langdetect
!pip install nltk
!pip install numpy
!pip install tqdm
!pip install transformers
!pip install torch

In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, MapType, TimestampType
# from pyspark.sql.types import *
# from pyspark.sql.functions import *
import zipfile
import pandas as pd
import numpy as np
from pyspark.sql.functions import col
import os
from pyspark.sql.functions import lit, to_date, to_timestamp
from pyspark.sql.functions import input_file_name

In [0]:
path_m = "/mnt/adls/test/PrachiSingh/SentAnalysis/FMCGCompany/Product_SKU.xlsx"

In [0]:
df_m = spark.read\
          .format("com.crealytics.spark.excel")\
          .option("header", "true")\
          .option("treatEmptyValuesAsNulls", "true")\
          .load(path_m)

In [0]:
# Converting the Pyspark Dataframe to Pandas Dataframe because I am using the later and related library for the analysis.

In [0]:
df_p = df_m.toPandas()

In [0]:
# Handling null values

In [0]:
df_sum = df_p.isnull().sum().to_frame() 

In [0]:
df_p.shape[0] 

In [0]:
df_sum.display()

In [0]:
# Dropping columns that contains nulls more than 30% of the records i.e, 155 records
# Col No. 79,98
df_p = df_p.drop(df_p.columns[[78,97]], 
                       axis = 1)

In [0]:
# Changing Yes and No to 1 and 0 respectively for easier EDA
df_p = df_p.replace(to_replace = "Yes", value = 1)
df_p = df_p.replace(to_replace = "No", value = 0)

In [0]:
# Filling the columns with ffill() function for those columns with null values less than 30%
# I found that the first row of the dataframe does not contain any nulls, so, it is safe to use the interpolate methd(interpolate(method ='linear', limit_direction ='forward')) for filling the null values.
df_p = df_p.interpolate(method ='linear', limit_direction ='forward')

In [0]:
df_p.info()

In [0]:
df_col_names = df_p.columns.to_frame() 

In [0]:
df_col_names.display()

In [0]:
df_p['Recommend_to_Others_on_0to10'] = df_p['Recommend_to_Others_on_0to10'].replace(['Extremely recommend'], '10')
df_p['Recommend_to_Others_on_0to10'] = df_p['Recommend_to_Others_on_0to10'].replace(['Not at all recommend'], '0')

In [0]:
# Sorting the dataframe according to the column- "Recommend_FMCG_Company_On_A_0_10_Scale"
# First convert column "Recommend_FMCG_Company_On_A_0_10_Scale" to numeric type and then sort.
df_p[["Recommend_to_Others_on_0to10"]] = df_p[["Recommend_to_Others_on_0to10"]].apply(pd.to_numeric)
df_p = df_p.sort_values(by=['Recommend_to_Others_on_0to10'], ascending=True)

In [0]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.tag import pos_tag
import seaborn as sns

In [0]:
nltk.download('punkt')
nltk.download('stopwords')

In [0]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

In [0]:
nltk.download('vader_lexicon')

In [0]:
sia = SentimentIntensityAnalyzer()

In [0]:
df_p[['Reason_for_rating']] = df_p[['Reason_for_rating']].astype(str)

In [0]:
# Running the polarity score on the entire dataset
res={}
for i, row in tqdm(df_p.iterrows(),total=len(df_p.columns)):
  text = row['Reason_for_rating']
  myid = row['Unique_ID_for_Respondent']
  res[myid]=sia.polarity_scores(text)

In [0]:
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index':'Unique_ID_for_Respondent'})
vaders = vaders.merge(df_p,how='left')

In [0]:
vaders = vaders.rename(columns={'Recommend_to_Others_on_0to10': 'Recommendation_Score'})

In [0]:
# Positive Score Vs Recommend Rating
sns.barplot(x = 'Recommendation_Score',
            y = 'pos',
            data = vaders)
plt.xlabel("Recommendation Score")
plt.ylabel("Positive Score from VADER")
plt.show()

In [0]:
# Neutral Score Vs Recommend Rating
sns.barplot(x = 'Recommendation_Score',
            y = 'neu',
            data = vaders)
plt.xlabel("Recommendation Score")
plt.ylabel("Neutral Score from VADER")
plt.show()

In [0]:
# Negative Score Vs Recommend Rating
sns.barplot(x = 'Recommendation_Score',
            y = 'neg',
            data = vaders)
plt.xlabel("Recommendation Score")
plt.ylabel("Negative Score from VADER")
plt.show()

In [0]:
# Compound Score Vs Recommend Rating
sns.barplot(x = 'Recommendation_Score',
            y = 'compound',
            data = vaders)
plt.xlabel("Recommendation Score")
plt.ylabel("Compound Score from VADER")
plt.show()

In [0]:
def f(row):
    if row['compound'] >= 0.5:
        val = 1 #'Positive Sentiment'
    elif (row['compound'] > -0.5) & (row['compound'] < 0.5):
        val = 2 #'Neutral Sentiment'
    elif row['compound'] <= -0.5:
        val = 3 #'Negative Sentiment'
    return val

In [0]:
vaders['Sentiment_Type'] = vaders.apply(f, axis=1)

In [0]:
# print(vaders["Sentiment_Type"].unique())

In [0]:
vaders['Sentiment_Type'].value_counts()

In [0]:
def f_RatingGroup(row):
    if (row['Recommendation_Score'] in [0,1,2,3,4,5,6,7]):
        return 'Bad' #1
    elif (row['Recommendation_Score'] in [8,9]):
        return 'Good' #2
    elif (row['Recommendation_Score'] in [10]):
        return 'Excellent' #3

In [0]:
vaders['Recommendation_Score_New'] = vaders.apply(f_RatingGroup, axis=1)

In [0]:
vaders['Recommendation_Score_New'].value_counts()

In [0]:
vaders = vaders.rename(columns={'Decides_on_the_Kitchen_purchase': 'Decides_on_the_Product_SKU_purchase'})

In [0]:
# This function will be utilised in a lot of analysis
def get_count_df(df):
  list_name=[]
  count_yes=[]
  for c in df.columns:
    list_name.append(c)
    count_yes.append(df[c].value_counts()[1])
  var_key = {'var_name':list_name,'count_value':count_yes}
  new_df = pd.DataFrame(var_key)
  new_df = new_df.sort_values(by=['count_value'], ascending=False)
  return new_df

In [0]:
df_channel = vaders.loc[:, ['Billboards_Signs','TV_Advertisement','Store_signage','Print_Ad_newspaper','Google_Ads','Google_my_business_store_page','Flyers_Pamphlets','Relatives_Friends','Website','Instagram','Facebook','YouTube',
'Walk_in_showroom']]

In [0]:
channel_df = get_count_df(df_channel)

In [0]:
list_channel_name = ["Billboards / Signs",	"TV Advertisement",	"Store signage",	"Print Ad (newspaper)",	"Google Ads",	"Google my business store page",	"Flyers / Pamphlets",	"Relatives / Friends",	"Website",	"Instagram",	"Facebook",	"YouTube",	"Walk in (showroom)"
]

In [0]:
channel_df['var_name'] = list_channel_name

In [0]:
channel_df["PercentageOfTotalInterviewee"] = channel_df["count_value"] *(100/670)

In [0]:
channel_df = channel_df.sort_values(by=['count_value'], ascending=False)

In [0]:
channel_df.plot.barh(x="var_name", y="count_value", title="Media Vs Count of customers")
plt.ylabel("Media")
plt.xlabel("Count of customers") 
plt.show()

In [0]:
df_age_marital_cust = vaders.loc[:, ['Gender','Age','MaritalStatus','Recommendation_Score']]

In [0]:
def f_RatingScore(row):
    if (row['Recommendation_Score'] in [0,1,2,3,4,5,6,7]):
        return 1
    elif (row['Recommendation_Score'] in [8,9]):
        return 2
    elif (row['Recommendation_Score'] in [10]):
        return 3

In [0]:
df_age_marital_cust['RatingScore'] = df_age_marital_cust.apply(f_RatingScore, axis=1)

In [0]:
df_age_marital_cust = df_age_marital_cust.drop(columns=['Recommendation_Score'])

In [0]:
# Changing the column type of age column for mathematical manipulation possible over it.
df_age_marital_cust["Age"] = pd.to_numeric(df_age_marital_cust["Age"])

In [0]:
def f_ageGroup(row):
    if row['Age'] <= 25:
        return 'Below 25'
    elif (row['Age'] > 25) & (row['Age'] < 35):
        return 'Between 25 and 35'
    elif (row['Age'] > 35) & (row['Age'] < 50):
        return 'Between 35 and 50'
    elif row['Age'] >= 50:
        return 'Above 50'

In [0]:
df_age_marital_cust['Age_Group'] = df_age_marital_cust.apply(f_ageGroup, axis=1)

In [0]:
df_age_marital_cust = df_age_marital_cust.sort_values(by=['Age','RatingScore'], ascending=True)

In [0]:
df_age_marital_cust = df_age_marital_cust.drop(columns=['Age'])

In [0]:
df_gender = df_age_marital_cust.groupby(["Gender", "RatingScore"])["RatingScore"].value_counts().unstack(fill_value=0)

In [0]:
df_gender.plot(kind="bar", stacked=True)
plt.xticks(rotation=90, horizontalalignment="center")
plt.title("Analysis of Gender of the Respondent and their Recommendation Score")
plt.show()

In [0]:
df_age = df_age_marital_cust.groupby(["Age_Group", "RatingScore"])["RatingScore"].value_counts().unstack(fill_value=0)

In [0]:
df_age.plot.bar(figsize = (10, 12), stacked=True)
plt.xticks(rotation=90, horizontalalignment="center")
plt.title("Analysis of Age group of the Respondent and their Recommendation Score")
plt.show()

In [0]:
df_marital = df_age_marital_cust.groupby(["MaritalStatus", "RatingScore"])["RatingScore"].value_counts().unstack(fill_value=0)

In [0]:
df_marital.plot(kind="bar", stacked=True)
plt.xticks(rotation=90, horizontalalignment="center")
plt.title("Analysis of Marital status of the Respondent and their Recommendation Score")
plt.show()

In [0]:
# This function will be utilised in a lot of analysis
# 'Poor':1, 'Fair':2, 'Good':3, 'Very Good':4, 'Excellent':5
def get_count_df_good(df):
  list_name=[]
  count_p=[]
  count_f=[]
  count_g=[]
  count_vg=[]
  count_e=[]
  for c in df.columns:
    list_name.append(c)
    if 'Poor' in df[c].unique().tolist(): 
      count_p.append(df[c].value_counts()['Poor'])
    else:
      count_p.append(0)
    if 'Fair' in df[c].unique().tolist(): 
      count_f.append(df[c].value_counts()['Fair'])
    else:
      count_f.append(0)
    if 'Good' in df[c].unique().tolist(): 
      count_g.append(df[c].value_counts()['Good'])
    else:
      count_g.append(0)
    if 'Very Good' in df[c].unique().tolist(): 
      count_vg.append(df[c].value_counts()['Very Good'])
    else:
      count_vg.append(0)
    if 'Excellent' in df[c].unique().tolist(): 
      count_e.append(df[c].value_counts()['Excellent'])
    else:
      count_e.append(0)
  var_key = {'var_name':list_name,'Poor':count_p,'Fair':count_f,'Good':count_g,'Very Good':count_vg,'Excellent':count_e}
  new_df = pd.DataFrame(var_key)
  return new_df

In [0]:
# Imagery
df_imagery_FMCG_Company = vaders.loc[:, ['FMCG_Company_offers_latest_trends_and_contemporary_designs_of_Product_SKU','FMCG_Company_is_very_reliable_to_buy_product_from','FMCG_Company_is_very_innovative_in_its_products_and_designs','FMCG_Company_offers_Product_SKU_which_promotes_health','FMCG_Company_is_a_company_which_understands_my_needs_and_wants','FMCG_Company_products_are_Durable_for_long-term_use','FMCG_Company_products_have_better_features_compared_to_other_brands'
]]

In [0]:
imagery_FMCG_Company_df = get_count_df_good(df_imagery_FMCG_Company)

In [0]:
# ax = showroom_FMCG_Company_df.plot.bar(stacked=True)
imagery_FMCG_Company_df.plot.bar(figsize = (10, 12))
# imagery_FMCG_Company_df.plot(kind="barh", stacked=True)
plt.title("Number of Customers Vs Experience with Imagery")
# plt.xticks(rotation=45, horizontalalignment="center")
plt.xlabel("Experience with Imagery")
plt.ylabel("Number of Customers")
plt.show()

In [0]:
# Showroom
df_showroom_FMCG_Company = vaders.loc[:, ['Accessibility_of_showroom','Ambience_of_the_showroom','Cleanliness','Variety_sufficient_adequate_display_in_the_store','Parking_convenience','Locating_store_through_Google_Maps','Availability_of_latest_e_catalogue','Range_of_products_displayed','Availability_of_designs_colour_palette_material_options_within_products_of_choice'
]]

In [0]:
showroom_FMCG_Company_df = get_count_df_good(df_showroom_FMCG_Company)

In [0]:
showroom_FMCG_Company_df.plot.bar(figsize = (10, 12))
plt.title("Number of Customers Vs Experience with Showroom")
plt.xlabel("Experience with Showroom")
plt.ylabel("Number of Customers")
plt.show()

In [0]:
df_staff = vaders.loc[:, ['Courteousness_Behaviour_of_staff','Personnel_was_knowledgeable_about_products','Ability_of_staff_to_address_your_queries','Ability_of_staff_to_suggest_provide_right_products_for_your_requirement','Greeted_you_as_soon_as_you_entered_the_store_or_were_approached_by_a_staff','Store_staff_ability_to_communicate_about_prod_price_offer','Communication_skills_of_staff','Grooming_of_the_showroom_personnel','Quality_of_recommendation_design_suggestion'
]]

In [0]:
staff_df = get_count_df_good(df_staff)

In [0]:
staff_df.plot.bar(figsize = (10, 12))
plt.title("Number of Customers Vs Experience with Staff Behaviour")
plt.xlabel("Experience with Staff Behaviour")
plt.ylabel("Number of Customers")
plt.show()

In [0]:
df_prod_purchase = vaders.loc[:, ['Product_Quality','Products_offered_are_different_unique_in_the_market','Durability_of_the_product','Comfort_of_the_Product_SKU'
]]

In [0]:
prod_purchase_df = get_count_df_good(df_prod_purchase)

In [0]:
prod_purchase_df.plot.bar(figsize = (10, 12))
plt.title("Number of Customers Vs Experience with Product Purchase")
plt.xlabel("Experience with Product Purchase")
plt.xlabel("Number of Customers")
plt.show()

In [0]:
df_Product_SKU_purchase = vaders.loc[:, ['Availability_of_range_in_product_miniature_for_all_kinds_of_material',
'Availability_of_exact_dimensions','Proactive_explanation_about_products'
]]

In [0]:
Product_SKU_purchase_df = get_count_df_good(df_Product_SKU_purchase)

In [0]:
Product_SKU_purchase_df.plot.bar(figsize = (10, 12))
plt.title("Number of Customers Vs Experience with Product_SKU Purchase")
plt.xlabel("Experience with Product_SKU Purchase")
plt.ylabel("Number of Customers")
plt.show()

In [0]:
df_billing = vaders.loc[:, ['Product_price','Discount_Offers_at_the_time_of_purchase','Clarity_of_communication_on_return_refund_replacement_policy_claim_policy','Time_taken_to_complete_purchase_formalities','Billing_time_taken','Payment_Terms','Warranty_and_Guarantee','Availability_of_Product_SKU_at_various_price_points'
]]

In [0]:
billing_df = get_count_df_good(df_billing)

In [0]:
billing_df.plot.bar(figsize = (10, 12))
plt.title("Number of Customers Vs Experience with Billing Experience")
plt.xlabel("Experience with Billing")
plt.ylabel("Number of Customers")
plt.show()

In [0]:
# Delivery

In [0]:
df_delivery = vaders.loc[:, ['Adherence_to_committed_delivery_time','Condition_of_product_on_delivery','Prior_intimation_of_delivery_delay_in_delivery','Dealer_personnel_was_professionally_groomed']]

In [0]:
delivery_df = get_count_df_good(df_delivery)

In [0]:
delivery_df.plot.bar(figsize = (10, 12))
plt.title("Number of Customers Vs Experience with Delivery")
plt.xlabel("Experience with Delivery")
plt.ylabel("Number of Customers")
plt.show()

In [0]:
# Overall Experience

In [0]:
df_overall = vaders.loc[:, ['Based_on_your_experience_you_are_satisfied_with_FMCG_Company','You_will_definitely_buy_FMCG_Company_for_your_future_needs','You_believe_that_FMCG_Company_delivers_the_best_value_for_the_money_you_have_spent','You_will_definitely_buy_other_products_and_services_from_FMCG_Company'
]]

In [0]:
overall_df = get_count_df_good(df_overall)

In [0]:
overall_df.plot.bar(figsize = (10, 12))
plt.title("Number of Customers Vs their Overall Experience")
plt.xlabel("Overall Experience of Customers")
plt.ylabel("Number of Customers")
plt.show()

In [0]:
# Rating given by customers 
df_rating = vaders.loc[:, ['Rate_Overall_Experience','Based_on_your_experience_you_are_satisfied_with_FMCG_Company','Rate_experience_on_the_Showroom','Rate_experience_with_the_Sales_Staff','Rate_experience_in_the_Purchase_of_FMCG_Company_Product_SKU','Rate_the_Product_SKU_purchase_experience_in_the_FMCG_Company_Product_SKU','Rate_the_Purchase_experience_price_discounts_billing_warranty','Rate_experience_with_the_Product_Delivery'
]]

In [0]:
rating_df = get_count_df_good(df_rating)

In [0]:
rating_df.plot.bar(figsize = (10, 12))
plt.title("Number of Customers Vs Ratings")
plt.xlabel("Ratings")
plt.ylabel("Number of Customers")
plt.show()

In [0]:
df_Rating_FMCG_Company_1 = df_rating

In [0]:
for c in df_Rating_FMCG_Company_1.columns:    
  df_Rating_FMCG_Company_1 = df_Rating_FMCG_Company_1.replace({c: {'Poor':1, 'Fair':2, 'Good':3, 'Very Good':4, 'Excellent':5}})

In [0]:
df_Rating_FMCG_Company_1.columns = ['0', '1', '2','3','4','5','6','7']

In [0]:
df_Rating_FMCG_Company_corr = df_Rating_FMCG_Company_1.corr(
    method = 'pearson',  # The method of correlation
    min_periods = 1      # Min number of observations required
)

In [0]:
sns.heatmap(df_Rating_FMCG_Company_corr,annot=True)
plt.show()